In [2]:
import ee

Autenticación estándar de GEE

In [3]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-NA6B-dZ-7SGKMQAwe_7jHayLPaQpjuLYYNilw70iLM&tc=IKHynfdcsIgd0y56erFniRZMDgFn57zZqhBb0XuikYM&cc=HfAgsHHiM1gnmRnocYZRCE47QOPQIL2weykzouUoqL8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdd_2fcRlqZEJlW5MmDdSvZTo1YQriCpuV5Rm8Kn3lqHXIlaMTe9-us

Successfully saved authorization token.


In [4]:
ee.Initialize()

Se solicita la ubicación del archivo con los puntos (placemarks.csv)

In [86]:
points = input("Ingrese la dirección de los puntos:")

Ingrese la dirección de los puntos:projects/lofty-optics-360613/assets/PuntoAnual


Se generan los datasets iniciales

In [87]:
table = ee.FeatureCollection(points)

# Las imágenes de cada punto se recortan en un cuadrado de 30x30 metros
# centradas en el punto correspondiente
bounds = table.geometry().buffer(60).bounds()

# Las imagenes de Sentinel se seleccionan en función de que su fecha de captura
# esté entre un mes antes y un mes después del período total que abarcan
# los puntos de interés
sentinel1 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(bounds) \
    .filterDate('2021-01-1', '2021-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))

Se preparan los datos de los puntos convirtiendo sus fechas a un objeto Date, y reemplazando los puntos por cuadrados de 30 metros de lado con centro en el punto correspondiente

In [88]:
# Función que reemplaza la propiedad 'Date' como string, por otra
# del mismo nombre representada por un objeto ee.Date
def func_qom (feature):
  ft =  feature.set('Date', ee.Date(feature.get('Date'))) \
                .set('ids', feature.get('ids'))
  return ee.Feature(ft)

In [89]:
placemarks = table.select(['Date', 'ids']).map(func_qom)

In [90]:
def get_snip(feature):
  return ee.Feature(feature.buffer(40).bounds())

In [91]:
snips = placemarks.map(get_snip).filter(ee.Filter.eq('ids',ee.String('N412B84')))

In [92]:
snips.size().getInfo()

1

Se realiza el procesamiento principal de los datos, generando los recortes deseados en función de los datasets de entrada y las fechas ingresadas

In [57]:
def generate_dataset_by_feature(features, images, window_start=0, window_end=0):
  """
  Genera un una ImageCollection donde cada imagen es un recorte de una imagen
  según su pertenencia a una Feature que representa los bordes del recorte
  y la fecha.
  features: ee.FeatureCollection donde cada feature es un polígono y contiene
  una propiedad 'Date' de tipo ee.Date
  images: ee.ImageCollection en la que las imágenes poseen fecha de captura
  window_start: numero entero que indica el primer día de la ventana de interés
  (a partir de qué día se considera que una imagen es relevante) respecto a la
  fecha de la Feature con la que se está comparando. 0 por defecto
  window_end: numero entero que indica el último día de la ventana de interés
  (a partir de qué día se considera que una imagen deja de ser relevante)
  respecto a la fecha de la Feature con la que se está comparando.
  0 por defecto.
  
  Ejemplo: window_start = -10, window_end = 10
  Por cada feature generará el recorte de todas las imágenes cuya fecha
  de captura esté entre 10 días antes y 10 días después del valor de Date
  de la Feature.
  """
  
  def by_feature(index):
    # Toma una Feature a partir de su propiedad 'system: index'
    ft = ee.Feature(
        features.filter(ee.Filter.eq('system:index', ee.String(index))).first()
        )
    id = ft.get('ids')
    
    # Produce una lista de los valores de 'system:index' de una ImageCollection
    # que resulta de filtrar otra ImageCollection en función de si su fecha
    # de captura se encuentra dentro de la ventana de interés respecto a la
    # Feature recuperada en el paso anterior
    time_window = images.filter(
        ee.Filter.date(
            ee.Date(ft.get('Date')).advance(window_start, 'day'),
            ee.Date(ft.get('Date')).advance(window_end, 'day')
            )
        ).filterBounds(ft.geometry()).aggregate_array('system:index')
    
    def func_uhc(index):
      # Toma una imagen a partir de su propiedad 'system:index' y la recorta
      # según los límites indicados por la Feature ft
      return ee.Image(
          images.filter(
            ee.Filter.eq(
              'system:index', ee.String(index)
              )
            ).first()
          ).clip(ft).set('ids', id)
    
    clipped = time_window.map(func_uhc)
    return clipped

  # Se obtiene una lista con los valores de la propiedad 'system:index'
  # de la colección features
  list = features.aggregate_array('system:index')

  # Se mapea la función by_feature sobre la lista recién generada
  results_matrix = list.map(by_feature)

  # El paso anterior genera una lista de listas, a continuación se genera una
  # ImageCollection a partir de la lista aplanada 
  results = ee.ImageCollection.fromImages(results_matrix.flatten())

  return results

Se exportan las imágenes generadas

In [94]:
from datetime import date

step = 30
begin = 0
end = 365

for day in range(begin, end, step):

  print(day)

  snips_collection = generate_dataset_by_feature(
      snips, sentinel1, day, day + step
      )
  
  collectionSize = snips_collection.size().getInfo()

  print(collectionSize)

  collectionList = snips_collection.toList(snips_collection.size())

  if(collectionSize>0):
    info = collectionList.getInfo()

  batch = []
  j = 0
  for i in range(0,1):

    
    print(i)
    img = ee.Image(collectionList.get(i)).select(['B4', 'B3', 'B2','B8'])
    img_30 = ee.Image(collectionList.get(i)).select(['B8A','B11','B12'])

    crs = info[i]['bands'][0]['crs']
    image_10m = img_30.resample('bilinear').reproject(crs=crs, scale=10)
    img = img.addBands(image_10m) 
    id = info[i]['properties']['ids']

    fecha = date.fromtimestamp(
        info[i]['properties']['system:time_start']/1000
        ).isoformat()

    batch.append(
      ee.batch.Export.image.toDrive(
        image = img, 
        folder = 'LaPlata_N412YB84',
        fileNamePrefix = f"LaPlata_{id}_{fecha}"
        )
      )
    batch[j].start()
    j+=1
    while(j==10):
      if batch[1].status()["state"] == "COMPLETED":
        j=0
        batch=[]

0
8
0
30
3
0
60
3
0
90
8
0
120
5
0
150
4
0
180
4
0
210
4
0
240
3
0
270
5
0
300
6
0
330
3
0
360
0
0


In [ ]:
# print(batch[1].status()["state"])